1. Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?

micro: сумма верных срабатываний обоих классов (True Positive и True Negative), деленная на общее количество объектов. Дает значение, равное accuracy.

macro: среднее арифметическое метрик, взятых отдельно для каждого класса.

weighted: взвешенное среднее метрик, взятых отдельно для каждого класса. То есть сумма произведений метрик и долей каждого класса.

micro усреднение полезно в случае идеального баланса классов. 

macro усреднение полезно в том случае, когда всем классам хочется дать одинаковый вес, вне зависимости от того, сколько их находится в датасете. Например, выявление мошеннических транзакций.

weighted усреднение лучше применять, когда каждому классу разумнее дать вес согласно количеству. Пример - задача классификации животных.

2. В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

Алгоритмы отличаются друг от друга реализацией алгоритма Boosted Trees. 

Перед обучением создаются пары признак-сплит для всех признаков. LightGBM использует градиентную одностороннюю выборку, Catboost - MVS, которая представляет собой взвешенную выборку стохастического градиентного бустинга. XGboost не использует методов взвешенной выборки.

Catboost строит сбалансированное дерево: на каждом уровне выбирается пара признак-сплит, которая приносит наименьшие потери и используется для всех узлов уровня. LightGBM использует рост дерева по листьям: выбирает лист, который минимизирует потери, что приводит к росту несбалансированного дерева. XGboost разделяет до заданного max_depth, затем обрезает дерево в обратном направлении и удаляет части, в которых нет положительного выигрыша. Кроме того, есть возможность выполнять рост дерева по листьям, как в LightGBM.

Catboost имеет два режима обработки пропущенных значений: min - пропущенный значения заполняются минимумом по признаку, max - максимальным. LightGBM и XGboost пропущенные значения заполняет так, чтобы уменьшить потери при каждом разбиении.

Catboost имеет два метода важности признаков: PredictionValuesChange и LossFunctionChange. Для каждого признака PredictionValuesChange показывает, насколько в среднем изменяется предсказание при изменении значения признака. Признак будет иметь большую важность, если изменение значения признака вызывает большое изменение в прогнозируемом значении. LossFunctionChange: для каждого признака значение представляет собой разницу между значением потерь модели с этим признаком и без него. Поскольку переобучение модели без одного из признаков требует больших вычислительных затрат, эта модель строится приблизительно, используя исходную модель с этим признаком, удаленным из всех деревьев в ансамбле. Для расчета важности этого признака требуется набор данных. LightGBM и XGBoost имеют два похожих метода: Gain, который представляет собой улучшение точности, приносимое признаком ветвям, на которых он находится, split (LightGBM) и Frequency/Weight (XGBoost), которые вычисляют относительное количество раз, когда определенный признак встречается во всех расщеплениях деревьев модели. Этот метод может быть необъективным для категориальных признаков с большим количеством категорий. У XGBoost есть еще один метод, Охват, который представляет собой относительное количество наблюдений, связанных с признаком. Для каждого признака мы подсчитываем количество наблюдений, используемых для выбора узла листа.

Для обработки категориальных признаков Catboost использует комбинацию одноточечного кодирования и расширенного среднего кодирования. Для признаков с малым количеством категорий используется одноточечное кодирование. Для остальных категориальных столбцов CatBoost использует эффективный метод кодирования, который похож на среднее кодирование, но с дополнительным механизмом, направленным на снижение переподгонки. LightGBM разделяет категориальные признаки, разбивая их категории на 2 подмножества. Основная идея заключается в том, чтобы сортировать категории в соответствии с целью обучения при каждом разбиении. XGBoost не имеет встроенного метода для категориальных признаков.